In [1]:
from cytograph import Algorithm, creates, requires, div0
from cytograph.visualization.scatter import scatterc, scattern
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
from typing import List
import shoji
import cytograph as cg
import cytograph.visualization as cgplot
import collections
from typing import List, Optional
import matplotlib.patheffects as PathEffects
import fastcluster
import igraph
import leidenalg as la
import scipy.cluster.hierarchy as hc
from hmmlearn import hmm
from openTSNE import TSNE
from pynndescent import NNDescent
from scipy.spatial.distance import pdist
from sklearn.decomposition import PCA
from tqdm import trange
import warnings
import pandas as pd
warnings.filterwarnings('ignore')
from cytograph.visualization import Colorizer
from matplotlib.lines import Line2D
import scipy as sci
import scanpy as sc

In [2]:
db = shoji.connect()

In [3]:
ws = db.builds.jesper.GBM_all.SL057_filtered

In [4]:
ws1 = db.builds.jesper.GBM_all.SL040

In [5]:
adata = ws.create_anndata()

In [6]:
adata.var = adata.var.set_index('Gene')

In [7]:
adata.var.index = adata.var.index.astype('object')

In [8]:
adata.var_names_make_unique()

In [9]:
adata.var.index.is_unique

True

In [10]:
sc.pp.normalize_total(adata)

In [11]:
sc.pp.log1p(adata)

In [12]:
ws1 = db.builds.jesper.GBM_all.SL040

In [13]:
adata40 = ws1.create_anndata()

In [14]:
adata40.var.index
adata40.var.Gene
adata40.var = adata40.var.set_index('Gene')
adata40.var.index = adata40.var.index.astype('object')
adata40.var_names_make_unique()
adata40.var.index.is_unique


True

In [15]:
sc.pp.normalize_total(adata40)
sc.pp.log1p(adata40)

In [16]:
cells = adata[adata.obs['ManualAnnotationSL057_new'] == 'Oligodendrocytes',:]
sc.tl.rank_genes_groups(cells, 'Zone', groups=['Central'], reference='Periphery', method="wilcoxon")
gene_rank_oligoCentralPeri = sc.get.rank_genes_groups_df(cells, group='Central')[['names','logfoldchanges','pvals', 'pvals_adj']]
cells = adata40[adata40.obs['ManualAnnotationSL040'] == 'Oligodendrocytes',:]
sc.tl.rank_genes_groups(cells, 'ManualAnnotationSL040_central_peri', groups=['Central'], reference='Periphery', method="wilcoxon")
gene_rank_Oligo40central = sc.get.rank_genes_groups_df(cells, group=['Central'])[['names','logfoldchanges','pvals', 'pvals_adj']]

In [18]:
cells = adata[adata.obs['ManualAnnotationSL057_new'] == 'Tumor',:]
sc.tl.rank_genes_groups(cells, 'Zone', groups=['Central'], reference='Periphery', method="wilcoxon")
gene_rank_tumorCentralPeri = sc.get.rank_genes_groups_df(cells, group='Central')[['names','logfoldchanges','pvals', 'pvals_adj']]
cells = adata40[adata40.obs['ManualAnnotationSL040'] == 'Tumor',:]
sc.tl.rank_genes_groups(cells, 'ManualAnnotationSL040_central_peri', groups=['Central'], reference='Periphery', method="wilcoxon")
gene_rank_tumor40central = sc.get.rank_genes_groups_df(cells, group=['Central'])[['names','logfoldchanges','pvals', 'pvals_adj']]

In [17]:
#Same calc SL040 and 57 in TAM
cells = adata[adata.obs['ManualAnnotationSL057_TAM'] == 'TAM',:]
sc.tl.rank_genes_groups(cells, 'Zone', groups=['Central'], reference='Periphery', method="wilcoxon")
gene_rank_TAMCentralPeri = sc.get.rank_genes_groups_df(cells, group='Central')[['names','logfoldchanges','pvals', 'pvals_adj']]
cells = adata40[adata40.obs['ManualAnnotationSL040_TAM'] == 'TAM',:]
sc.tl.rank_genes_groups(cells, 'ManualAnnotationSL040_central_peri', groups=['Central'], reference='Periphery', method="wilcoxon")
gene_rank_TAM40central = sc.get.rank_genes_groups_df(cells, group=['Central'])[['names','logfoldchanges','pvals', 'pvals_adj']]

In [19]:
%%time
Genes = np.array(adata.var.index)
Rank = []
Rank2 = []
for x in Genes:
    A = np.where(x == np.array(gene_rank_Oligo40central['names']))[0][0]
    B = np.where(x == np.array(gene_rank_oligoCentralPeri['names']))[0][0]
    C = np.where(x == np.array(gene_rank_tumor40central['names']))[0][0]
    D = np.where(x == np.array(gene_rank_tumorCentralPeri['names']))[0][0]
    E = np.where(x == np.array(gene_rank_TAM40central['names']))[0][0]
    F = np.where(x == np.array(gene_rank_TAMCentralPeri['names']))[0][0]
    Rank.append(np.mean([A, B, C, D, E, F]))
    Rank2.append([A, B, C, D, E, F])

CPU times: user 20min 4s, sys: 1.04 s, total: 20min 5s
Wall time: 20min 9s


In [20]:
#With OPC-tumor cells comp
rank = dict(zip(Rank, Genes))
sorted_rank2 = dict(sorted(rank.items()))
sorted_rank2

{13.0: 'VIM',
 15.166666666666666: 'APOE',
 15.833333333333334: 'GAPDH',
 17.166666666666668: 'MT2A',
 27.0: 'LDHA',
 31.666666666666668: 'MT-RNR1',
 34.5: 'LGALS1',
 42.666666666666664: 'TIMP1',
 44.0: 'S100A6',
 55.166666666666664: 'PKM',
 58.0: 'CHI3L1',
 63.5: 'ALDOA',
 64.66666666666667: 'SPP1',
 76.16666666666667: 'ANXA2',
 83.66666666666667: 'TPI1',
 85.16666666666667: 'CD63',
 100.33333333333333: 'APOC1',
 104.83333333333333: 'MT-RNR2',
 139.0: 'SEC61G',
 158.16666666666666: 'MYL6',
 171.16666666666666: 'PGK1',
 177.16666666666666: 'C1QB',
 179.66666666666666: 'HMOX1',
 180.0: 'CTSB',
 181.16666666666666: 'CD44',
 187.66666666666666: 'ACTB',
 196.66666666666666: 'PFN1',
 206.66666666666666: 'VEGFA',
 208.33333333333334: 'MTATP6P1',
 210.5: 'SOCS3',
 220.66666666666666: 'CLU',
 247.83333333333334: 'TAGLN2',
 252.16666666666666: 'MT1X',
 252.83333333333334: 'TMBIM6',
 256.6666666666667: 'NUPR1',
 259.5: 'GUK1',
 272.5: 'COPE',
 276.1666666666667: 'SERF2',
 289.5: 'PRDX4',
 297.66

In [21]:
Genes2 = list(sorted_rank2.values())
Genes2

['VIM',
 'APOE',
 'GAPDH',
 'MT2A',
 'LDHA',
 'MT-RNR1',
 'LGALS1',
 'TIMP1',
 'S100A6',
 'PKM',
 'CHI3L1',
 'ALDOA',
 'SPP1',
 'ANXA2',
 'TPI1',
 'CD63',
 'APOC1',
 'MT-RNR2',
 'SEC61G',
 'MYL6',
 'PGK1',
 'C1QB',
 'HMOX1',
 'CTSB',
 'CD44',
 'ACTB',
 'PFN1',
 'VEGFA',
 'MTATP6P1',
 'SOCS3',
 'CLU',
 'TAGLN2',
 'MT1X',
 'TMBIM6',
 'NUPR1',
 'GUK1',
 'COPE',
 'SERF2',
 'PRDX4',
 'CIB1',
 'FTH1',
 'EIF4A1',
 'TXNDC17',
 'MYDGF',
 'GNG5',
 'IGFBP2',
 'PHPT1',
 'MIF',
 'FTL',
 'P4HB',
 'C4orf3',
 'SOD2',
 'DAD1',
 'ENO1',
 'TMED9',
 'NDUFS5',
 'ATP6V0E1',
 'PSMD8',
 'SERPINA3',
 'PPIB',
 'BCAP31',
 'PRELID1',
 'SPATA13',
 'TNFRSF12A',
 'TMEM208',
 'PSME2',
 'CFL1',
 'B2M',
 'SLC2A3',
 'SH3BGRL3',
 'APOC2',
 'TMSB10',
 'VAMP5',
 'HSPE1',
 'CHCHD2',
 'LUCAT1',
 'C1QC',
 'MAP1B',
 'SERPINE1',
 'MYL12A',
 'TXN',
 'UBL5',
 'MT-ND4L',
 'LAPTM4A',
 'METRN',
 'OAZ1',
 'PRDX5',
 'EIF5A',
 'CTSZ',
 'PSMA7',
 'ARPC1B',
 'GBP2',
 'VDAC1',
 'S100A10',
 'POMP',
 'C1QA',
 'HSPA5',
 'NUCB1',
 'BSG',
 'BA

In [22]:
Genes2 = np.array(Genes2)
Genes2

array(['VIM', 'APOE', 'GAPDH', ..., 'DDX17', 'CLASP2', 'FCHSD2'],
      dtype='<U22')

In [23]:
Top500 = Genes2[:500]

In [24]:
Top500

array(['VIM', 'APOE', 'GAPDH', 'MT2A', 'LDHA', 'MT-RNR1', 'LGALS1',
       'TIMP1', 'S100A6', 'PKM', 'CHI3L1', 'ALDOA', 'SPP1', 'ANXA2',
       'TPI1', 'CD63', 'APOC1', 'MT-RNR2', 'SEC61G', 'MYL6', 'PGK1',
       'C1QB', 'HMOX1', 'CTSB', 'CD44', 'ACTB', 'PFN1', 'VEGFA',
       'MTATP6P1', 'SOCS3', 'CLU', 'TAGLN2', 'MT1X', 'TMBIM6', 'NUPR1',
       'GUK1', 'COPE', 'SERF2', 'PRDX4', 'CIB1', 'FTH1', 'EIF4A1',
       'TXNDC17', 'MYDGF', 'GNG5', 'IGFBP2', 'PHPT1', 'MIF', 'FTL',
       'P4HB', 'C4orf3', 'SOD2', 'DAD1', 'ENO1', 'TMED9', 'NDUFS5',
       'ATP6V0E1', 'PSMD8', 'SERPINA3', 'PPIB', 'BCAP31', 'PRELID1',
       'SPATA13', 'TNFRSF12A', 'TMEM208', 'PSME2', 'CFL1', 'B2M',
       'SLC2A3', 'SH3BGRL3', 'APOC2', 'TMSB10', 'VAMP5', 'HSPE1',
       'CHCHD2', 'LUCAT1', 'C1QC', 'MAP1B', 'SERPINE1', 'MYL12A', 'TXN',
       'UBL5', 'MT-ND4L', 'LAPTM4A', 'METRN', 'OAZ1', 'PRDX5', 'EIF5A',
       'CTSZ', 'PSMA7', 'ARPC1B', 'GBP2', 'VDAC1', 'S100A10', 'POMP',
       'C1QA', 'HSPA5', 'NUCB1', 'BSG'